In [1]:
import pandas as pd
import json
from tqdm.notebook import trange, tqdm
pd.set_option('display.max_columns', None)
from datetime import date
import requests
import time
# import numpy as np

# df = pd.DataFrame({"id": ['1', '2', '3', '4', '5', '6', '7', '8'],

#                     "prepro": ['prepresp1', 'prepre1', 'prepresub1', 'prepresp2', 'prepre2', 'prepresub2', 'prepro1', 'prepro2'],

#                     "insert": [np.nan, np.nan, 'insert', np.nan, 'insert', np.nan, np.nan,  'insertcool'],

#                    "pre": ['presp1', 'presp1', 'presub1', 'presp2', 'presp2', 'presub2', np.nan, np.nan],

#                    "sp": ['sp1', np.nan, 'spsub1', 'sp2', np.nan, 'spsub2', np.nan, np.nan] ,

#                    "sub": [np.nan, np.nan, 'sub1', np.nan, np.nan, 'sub2', np.nan, np.nan]})

# df

In [2]:
observations_path = 'data/observations-285432.csv'
start_date = date(2022,9,1)
# start_date = 'min'
# finish_date = date(2019,5,30)
finish_date = 'max'

In [3]:
def prepare_df(observations_path, start_date, finish_date):
    
    df_full = pd.read_csv(observations_path)
    df = df_full.loc[:,'taxon_id':'taxon_form_name'].copy()
    df.insert(0,'created_at','')
    df['created_at'] = pd.to_datetime(df_full['created_at']).dt.date
    del(df_full)
    start_date = min(df['created_at']) if start_date == 'min' else start_date
    finish_date = max(df['created_at']) if finish_date == 'max' else finish_date
    return df, start_date, finish_date

In [4]:
df_taxons, start_date, finish_date = prepare_df(observations_path, start_date, finish_date)

In [5]:
df_taxons

,created_at,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_subphylum_name,taxon_superclass_name,taxon_class_name,taxon_subclass_name,taxon_superorder_name,taxon_order_name,taxon_suborder_name,taxon_superfamily_name,taxon_family_name,taxon_subfamily_name,taxon_supertribe_name,taxon_tribe_name,taxon_subtribe_name,taxon_genus_name,taxon_genushybrid_name,taxon_species_name,taxon_hybrid_name,taxon_subspecies_name,taxon_variety_name,taxon_form_name
0,2018-09-24,1062676,Fungi,Ascomycota,Pezizomycotina,NaN,Pezizomycetes,NaN,NaN,Pezizales,NaN,NaN,Morchellaceae,NaN,NaN,NaN,NaN,Morchella,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-24,1070919,Animalia,Arthropoda,Chelicerata,NaN,Arachnida,NaN,NaN,Araneae,Araneomorphae,Salticoidea,Salticidae,Salticinae,NaN,Sitticini,NaN,Attulus,NaN,Attulus floricola,NaN,NaN,NaN,NaN
2,2019-05-26,373470,Animalia,Arthropoda,Hexapoda,NaN,Insecta,Pterygota,NaN,Coleoptera,Polyphaga,Cleroidea,Melyridae,Malachiinae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-05-28,966205,Plantae,Tracheophyta,Angiospermae,NaN,Magnoliopsida,NaN,NaN,Fagales,NaN,NaN,Betulaceae,NaN,NaN,NaN,NaN,Alnus,NaN,Alnus glutinosa,NaN,NaN,NaN,NaN
4,2019-05-28,51610,Plantae,Tracheophyta,Angiospermae,NaN,Magnoliopsida,NaN,NaN,Lamiales,NaN,NaN,Plantaginaceae,NaN,NaN,Veroniceae,NaN,Veronica,NaN,Veronica chamaedrys,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
931,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_taxons_dict_to_date(df, date_to):

    df = df_taxons[df_taxons['created_at'] <= date_to].copy()
    df.drop('created_at', axis=1, inplace=True)
    
    taxons_full = df.columns
    df.dropna(axis=1, how='all', inplace=True)
    not_empty_taxons = df.columns

    last_levels = df.apply(lambda x: x.last_valid_index(), axis=1)
    levels = df.columns[df.columns.isin(last_levels.unique())].to_list()
    df = df[['taxon_id']+levels].copy()

    empty_taxons = set(taxons_full) - set(not_empty_taxons)
    full_taxons = set(not_empty_taxons) - set(df.columns)

    col_qty = len(df.columns)
    last_col = df.columns[col_qty-1]

    taxons_dict = dict()

    for i in range(1, col_qty):
        level_name = df.columns[i]
        taxons_dict[level_name] = dict()

    for i in range(1, col_qty):
        level_name = df.columns[i]
        level_taxons = df[last_levels == level_name][level_name].to_list()
        level_ids = df[last_levels == level_name].iloc[:,0].to_list()
        taxons_dict[level_name].update(zip(level_ids, level_taxons))
        i += 1

    return taxons_dict, df, empty_taxons, full_taxons

In [7]:
taxons_dict_start, df_taxons_clear_start, empty_taxons_start, full_taxons_start = get_taxons_dict_to_date(df_taxons, start_date)
taxons_dict_finish, df_taxons_clear_finish, empty_taxons_finish, full_taxons_finish = get_taxons_dict_to_date(df_taxons, finish_date)

In [8]:
def taxon_dict_to_df(taxons_dict):
    df = pd.DataFrame(columns=['taxon_id','taxon','taxon_name'])
    i = 0
    for taxon in taxons_dict.keys():
        for taxon_id in taxons_dict[taxon].keys():
            df.loc[i] = ([taxon_id, taxon, taxons_dict[taxon][taxon_id]])
            i += 1
    return df
    

In [9]:
taxons_df_start = taxon_dict_to_df(taxons_dict_start)
taxons_df_finish = taxon_dict_to_df(taxons_dict_finish)

In [10]:
taxons_df_finish

,taxon_id,taxon,taxon_name
0,47170,taxon_kingdom_name,Fungi
1,50814,taxon_class_name,Agaricomycetes
2,48372,taxon_class_name,Bryopsida
3,40151,taxon_class_name,Mammalia
4,1094814,taxon_subclass_name,Agaricomycetidae
...,...,...,...
501,55366,taxon_hybrid_name,Fragaria × ananassa
502,234770,taxon_subspecies_name,Calystegia sepium americana
503,448080,taxon_subspecies_name,Centaurea phrygia pseudophrygia
504,1397747,taxon_subspecies_name,Calystegia sepium spectabilis


In [11]:
def update_radius(taxon_list, radius, date_to):
    
    # получает список таксонов и радиус и дату, 
    # запрашивает у айнат, 
    # сохраняет в csv,
    # возвращает датафрейм с четыремя столбцами по запросу от айнат

    df = pd.DataFrame(columns=['taxon_id','radius','date','quantity'])

    radius = str(radius)
    date_to = str(date_to)
    url = 'https://api.inaturalist.org/v1/observations'
    i = 0
    for i in trange(len(taxon_list)):
        taxon_id = taxon_list[i]
        params = {
        'verifiable':'true',
        'taxon_id':taxon_id,
        'd2':date_to,
        'lat':'55.494403',
        'lng':'38.644662',
        'radius':radius,
        'order':'desc',
        'order_by':'created_at',
        'only_id':'true'
        }
        response = requests.get(url=url, params=params)
        quantity = response.json()['total_results']
        df.loc[i] = [taxon_id, radius, date_to, quantity]
        if response.status_code != 200:
            raise Exception('Oh response is not 200, it is ',response.status_code) 
        i += 1
        time.sleep(1)
        
    return df

In [12]:
radius = 200
date_to = date(2022,8,31)
radius = str(radius)
date_to = str(date_to)
url = 'https://api.inaturalist.org/v1/observations'
params = {
'verifiable':'true',
'taxon_id':'47170',
'd2':date_to,
'lat':'55.494403',
'lng':'38.644662',
'radius':radius,
'order':'desc',
'order_by':'created_at',
'only_id':'true'
}
response = requests.get(url=url, params=params)

In [13]:
response.json()['total_results']
# response.status_code

58910

In [14]:
taxon_list = taxons_df_finish['taxon_id'][0:10]
taxon_list

0      47170
1      50814
2      48372
3      40151
4    1094814
5     848317
6      48427
7      55484
8      52380
9      48195
Name: taxon_id, dtype: int64

In [15]:
# taxon_list = taxons_df_finish['taxon_id'][0:10]
# date_to = date(2022,8,31)
# first_radiuses = update_radius(taxon_list=taxon_list, radius=200, date_to=date_to)
# first_radiuses.to_csv(path_or_buf='data/radiuses_dataset.csv',index=False)

In [56]:
df = pd.read_csv(index_col=False, filepath_or_buffer='data/radiuses_dataset.csv')
saved_radiuses = df.copy()
ask_radiuses = df.copy()
ask_radiuses.drop('quantity', axis=1, inplace=True)
# ask_radiuses.insert(3, 'quantity', '')
saved_radiuses.iloc[0] = ['500','200','2023-01-25', '100']
saved_radiuses.iloc[2] = ['48372','200','2023-01-25', '100']
saved_radiuses.iloc[3] = ['48372','555','2023-01-25', '100']
saved_radiuses.iloc[5] = ['848317','200','2023-12-31', '100']
ask_radiuses

,taxon_id,radius,date
0,47170,200,2022-08-31
1,50814,200,2022-08-31
2,48372,200,2022-08-31
3,40151,200,2022-08-31
4,1094814,200,2022-08-31
5,848317,200,2022-08-31
6,48427,200,2022-08-31
7,55484,200,2022-08-31
8,52380,200,2022-08-31
9,48195,200,2022-08-31


In [57]:
saved_radiuses

,taxon_id,radius,date,quantity
0,500,200,2023-01-25,100
1,50814,200,2022-08-31,39634
2,48372,200,2023-01-25,100
3,48372,555,2023-01-25,100
4,1094814,200,2022-08-31,22023
5,848317,200,2023-12-31,100
6,48427,200,2022-08-31,3269
7,55484,200,2022-08-31,510
8,52380,200,2022-08-31,717
9,48195,200,2022-08-31,192


In [58]:
ask_radiuses

,taxon_id,radius,date
0,47170,200,2022-08-31
1,50814,200,2022-08-31
2,48372,200,2022-08-31
3,40151,200,2022-08-31
4,1094814,200,2022-08-31
5,848317,200,2022-08-31
6,48427,200,2022-08-31
7,55484,200,2022-08-31
8,52380,200,2022-08-31
9,48195,200,2022-08-31


In [65]:
csv_radiuses = pd.merge(ask_radiuses, saved_radiuses, how='left', left_on=['taxon_id','radius','date'], right_on=['taxon_id','radius','date'])
csv_radiuses

,taxon_id,radius,date,quantity
0,47170,200,2022-08-31,NaN
1,50814,200,2022-08-31,39634
2,48372,200,2022-08-31,NaN
3,40151,200,2022-08-31,NaN
4,1094814,200,2022-08-31,22023
5,848317,200,2022-08-31,NaN
6,48427,200,2022-08-31,3269
7,55484,200,2022-08-31,510
8,52380,200,2022-08-31,717
9,48195,200,2022-08-31,192


In [ ]:
def get_radius(taxon_list, radius, date_to):
    # получает список таксонов, радиус, дату
    # проверяет, какие радиусы есть в csv, каких нет - запрашивает у update_radius, какие есть - берёт из csv, отдаёт  
    # возвращает датафрейм с тремя столбцами по данным от iNat или из файла
    df = pd.read_csv(index_col=False, filepath_or_buffer='data/radiuses_dataset.csv')
    return df

df = get_radius(taxon_list, first_radiuses, date_to)
df

,taxon_id,radius,date,response
0,47170,200,2022-08-31,58910
1,50814,200,2022-08-31,39634
2,48372,200,2022-08-31,5463
3,40151,200,2022-08-31,9138
4,1094814,200,2022-08-31,22023
5,848317,200,2022-08-31,9027
6,48427,200,2022-08-31,3269
7,55484,200,2022-08-31,510
8,52380,200,2022-08-31,717
9,48195,200,2022-08-31,192
